# Functional Data Gathering - Layers

## Import modules

In [1]:
from caveclient import CAVEclient
import nglui
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from microns_phase3 import nda, utils
from standard_transform import minnie_transform_vx
tform_vx = minnie_transform_vx()

Connecting microns@db.datajoint.com:3306


## Building the dataframe of all functional cells

In [2]:
client = CAVEclient()

client.auth.save_token(token="a80f7690d93ead1b46e980628df5beae",overwrite = True)

# this is the datastack name of the public release
# passing it will auto-configure many of the services
client = CAVEclient('minnie65_public_v343')

#visualize available tables
client.materialize.get_tables()

['synapses_pni_2',
 'nucleus_neuron_svm',
 'nucleus_detection_v0',
 'functional_coreg',
 'allen_v1_column_types_slanted',
 'aibs_column_nonneuronal',
 'func_unit_em_match_release',
 'proofreading_status_public_release',
 'aibs_soma_nuc_metamodel_preds_v117']

In [3]:
df=client.materialize.query_table('functional_coreg') #dataframe of the functionally co-registered neurons
df.head() #visualize first couple neurons

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,pt_position
0,1,t,102531727972419182,864691136136642173,4,7,665,"[274864, 94064, 22046]"
1,2,t,94792540099826672,864691136208993724,4,7,740,"[218320, 104320, 19016]"
2,4,t,93946810036875732,864691135683934135,4,7,782,"[212128, 94544, 23423]"
3,5,t,84730565597057981,864691135334528233,4,7,855,"[145158, 110036, 19355]"
4,6,t,110905058841147285,864691136309730650,4,7,921,"[335744, 90128, 22805]"


In [4]:
#This transform function uses the pt_position column to give the depth value measured in microns
#this will allow us to select neurons based on their layer membership
transform = tform_vx.apply(df['pt_position'])
matrix_stacked = np.vstack(transform)
depths = matrix_stacked[:,1]

In [5]:
#This final transformation gives us the dataframe with the depths of every neuron listed in the pt_position column
df['pt_position'] = depths
df = df.rename(columns={'pt_position': 'depth'})
df.head()

,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,depth
0,1,t,102531727972419182,864691136136642173,4,7,665,73.977104
1,2,t,94792540099826672,864691136208993724,4,7,740,95.132458
2,4,t,93946810036875732,864691135683934135,4,7,782,54.018588
3,5,t,84730565597057981,864691135334528233,4,7,855,92.403500
4,6,t,110905058841147285,864691136309730650,4,7,921,79.517182


## Layer 1 Gathering

In [22]:
#creating dataframe with filtered depths values. This ensures we have only neurons that are in layer 1
df_layer1 = df.loc[(df['depth'] <= 98) & (df['session'] != 9)]
df_layer1

,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,depth
0,1,t,102531727972419182,864691136136642173,4,7,665,73.977104
1,2,t,94792540099826672,864691136208993724,4,7,740,95.132458
2,4,t,93946810036875732,864691135683934135,4,7,782,54.018588
3,5,t,84730565597057981,864691135334528233,4,7,855,92.403500
4,6,t,110905058841147285,864691136309730650,4,7,921,79.517182
...,...,...,...,...,...,...,...,...
9408,7350,t,98661997469563322,864691136966012366,6,7,1624,80.695285
9446,4967,t,99084620775287161,864691135476412328,6,7,964,93.419075
9468,6062,t,97254553866910822,864691135502353077,6,6,1872,74.226450
9492,1340,t,91625740990180053,864691135502369205,6,7,1161,82.094494


This loop iterates over the layer 1 dataframe taking each neurons identification and using it as an input into the fetch_oracle_raster method.
This method returns a 3d array with 10 repititions of 6 stimulus trials. Each trial contains 63 frames.

The array of activity (a 2d array organized as 6 arrays of 63 activity responses) for each repeat is then flattened and appended to its respective list to later be turned into dataframes.

The result will be 10 dataframes, each of size 378 x n with n being the number of neurons in the layer, for each layer. One for each repitition.

In [42]:
l1t1, l1t2, l1t3, l1t4, l1t5, l1t6, l1t7, l1t8, l1t9, l1t10 = ([] for i in range(10))
for row in range(len(df_layer1)):
    unit_key = {'session': df_layer1.iloc[row]['session'], 'scan_idx': df_layer1.iloc[row]['scan_idx'], 'unit_id': df_layer1.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    l1t1.append(oracle_traces[0].flatten())
    l1t2.append(oracle_traces[1].flatten())
    l1t3.append(oracle_traces[2].flatten())
    l1t4.append(oracle_traces[3].flatten())
    l1t5.append(oracle_traces[4].flatten())
    l1t6.append(oracle_traces[5].flatten())
    l1t7.append(oracle_traces[6].flatten())
    l1t8.append(oracle_traces[7].flatten())
    l1t9.append(oracle_traces[8].flatten())
    l1t10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [57]:
l1t1= np.array(l1t1)
l1t2= np.array(l1t2)
l1t3= np.array(l1t3)
l1t4= np.array(l1t4)
l1t5= np.array(l1t5)
l1t6= np.array(l1t6)
l1t7= np.array(l1t7)
l1t8= np.array(l1t8)
l1t9= np.array(l1t9)
l1t10= np.array(l1t10)
np.savetxt('layer1_trial1.csv', l1t1, delimiter= ',')
np.savetxt('layer1_trial2.csv', l1t2, delimiter= ',')
np.savetxt('layer1_trial3.csv', l1t3, delimiter= ',')
np.savetxt('layer1_trial4.csv', l1t4, delimiter= ',')
np.savetxt('layer1_trial5.csv', l1t5, delimiter= ',')
np.savetxt('layer1_trial6.csv', l1t6, delimiter= ',')
np.savetxt('layer1_trial7.csv', l1t7, delimiter= ',')
np.savetxt('layer1_trial8.csv', l1t8, delimiter= ',')
np.savetxt('layer1_trial9.csv', l1t9, delimiter= ',')
np.savetxt('layer1_trial10.csv', l1t10, delimiter= ',')


## Layer 2/3 Gathering

In [6]:
df_layer23 = df[df['depth'].between(98, 283) & (df['session'] != 9)]
df_layer23

,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,depth
8,12,t,115623132022390280,864691136521587601,4,7,2585,191.044803
9,13,t,115623201144775522,864691135593561643,4,7,2591,192.849116
10,15,t,110347193524762590,864691135382480602,4,7,2621,229.130676
11,17,t,111754568542850272,864691135467558284,4,7,2626,232.799003
12,18,t,110277031073449184,864691135683238898,4,7,2647,235.657765
...,...,...,...,...,...,...,...,...
9512,4646,t,103799739958717633,864691135654475202,5,7,2482,117.643300
9514,8479,t,96484208197075426,864691135569353862,7,3,1827,184.137968
9515,7837,t,86631484635009955,864691135925955086,5,6,2592,125.911236
9516,8718,t,96906420528106308,864691135082076407,8,5,1859,184.970331


In [24]:
l23t1, l23t2, l23t3, l23t4, l23t5, l23t6, l23t7, l23t8, l23t9, l23t10 = ([] for i in range(10))
for row in range():
    unit_key = {'session': df_layer23.iloc[row]['session'], 'scan_idx': df_layer23.iloc[row]['scan_idx'], 'unit_id': df_layer23.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    l23t1.append(oracle_traces[0].flatten())
    l23t2.append(oracle_traces[1].flatten())
    l23t3.append(oracle_traces[2].flatten())
    l23t4.append(oracle_traces[3].flatten())
    l23t5.append(oracle_traces[4].flatten())
    l23t6.append(oracle_traces[5].flatten())
    l23t7.append(oracle_traces[6].flatten())
    l23t8.append(oracle_traces[7].flatten())
    l23t9.append(oracle_traces[8].flatten())
    l23t10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [25]:
l23t1= np.array(l23t1)
l23t2= np.array(l23t2)
l23t3= np.array(l23t3)
l23t4= np.array(l23t4)
l23t5= np.array(l23t5)
l23t6= np.array(l23t6)
l23t7= np.array(l23t7)
l23t8= np.array(l23t8)
l23t9= np.array(l23t9)
l23t10= np.array(l23t10)
np.savetxt('layer23_trial1_2001_to_4147.csv', l23t1, delimiter= ',')
np.savetxt('layer23_trial2_2001_to_4147.csv', l23t2, delimiter= ',')
np.savetxt('layer23_trial3_2001_to_4147.csv', l23t3, delimiter= ',')
np.savetxt('layer23_trial4_2001_to_4147.csv', l23t4, delimiter= ',')
np.savetxt('layer23_trial5_2001_to_4147.csv', l23t5, delimiter= ',')
np.savetxt('layer23_trial6_2001_to_4147.csv', l23t6, delimiter= ',')
np.savetxt('layer23_trial7_2001_to_4147.csv', l23t7, delimiter= ',')
np.savetxt('layer23_trial8_2001_to_4147.csv', l23t8, delimiter= ',')
np.savetxt('layer23_trial9_2001_to_4147.csv', l23t9, delimiter= ',')
np.savetxt('layer23_trial10_2001_to_4147.csv', l23t10, delimiter= ',')


## Layer 4 Gathering

In [24]:
df_layer4 = df[df['depth'].between(283, 371) & (df['session'] != 9)]
df_layer4

,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,depth
85,105,t,115205455311342003,864691136482891308,4,7,5397,324.596409
86,106,t,112531649056527860,864691135064986436,4,7,5454,324.704876
87,107,t,111969111621450168,864691135396379681,4,7,5459,334.617590
88,108,t,109717860690956483,864691136134446475,4,7,5466,344.816976
89,109,t,110492054517316767,864691136008426668,4,7,5471,351.690073
...,...,...,...,...,...,...,...,...
9494,9331,t,109506549374057551,864691135776167085,4,7,5516,339.163784
9496,4513,t,117666917846061069,864691134917699594,5,3,4286,288.311902
9502,8237,t,86569154861620093,864691135815842895,6,4,7030,363.889519
9504,7462,t,90085668931340969,864691135994851242,7,4,8968,316.629869


In [ ]:
l4t1, l4t2, l4t3, l4t4, l4t5, l4t6, l4t7, l4t8, l4t9, l4t10 = ([] for i in range(10))
for row in range(len(df_layer4)):
    unit_key = {'session': df_layer4.iloc[row]['session'], 'scan_idx': df_layer4.iloc[row]['scan_idx'], 'unit_id': df_layer4.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    l4t1.append(oracle_traces[0].flatten())
    l4t2.append(oracle_traces[1].flatten())
    l4t3.append(oracle_traces[2].flatten())
    l4t4.append(oracle_traces[3].flatten())
    l4t5.append(oracle_traces[4].flatten())
    l4t6.append(oracle_traces[5].flatten())
    l4t7.append(oracle_traces[6].flatten())
    l4t8.append(oracle_traces[7].flatten())
    l4t9.append(oracle_traces[8].flatten())
    l4t10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [ ]:
l4t1= np.array(l4t1)
l4t2= np.array(l4t2)
l4t3= np.array(l4t3)
l4t4= np.array(l4t4)
l4t5= np.array(l4t5)
l4t6= np.array(l4t6)
l4t7= np.array(l4t7)
l4t8= np.array(l4t8)
l4t9= np.array(l4t9)
l4t10= np.array(l4t10)
np.savetxt('layer4_trial1_1_to_1340.csv', l4t1, delimiter= ',')
np.savetxt('layer4_trial2_1_to_1340.csv', l4t2, delimiter= ',')
np.savetxt('layer4_trial3_1_to_1340.csv', l4t3, delimiter= ',')
np.savetxt('layer4_trial4_1_to_1340.csv', l4t4, delimiter= ',')
np.savetxt('layer4_trial5_1_to_1340.csv', l4t5, delimiter= ',')
np.savetxt('layer4_trial6_1_to_1340.csv', l4t6, delimiter= ',')
np.savetxt('layer4_trial7_1_to_1340.csv', l4t7, delimiter= ',')
np.savetxt('layer4_trial8_1_to_1340.csv', l4t8, delimiter= ',')
np.savetxt('layer4_trial9_1_to_1340.csv', l4t9, delimiter= ',')
np.savetxt('layer4_trial10_1_to_1340.csv', l4t10, delimiter= ',')


## Layer 5 Gathering

In [27]:
df_layer5 = df[df['depth'].between(371, 574) & (df['session'] != 9)]
df_layer5

,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,depth
56,2486,t,113663045917715038,864691135946685793,4,7,8034,489.940524
66,2568,t,88049510366229524,864691135303388583,5,3,7019,444.995864
163,175,t,114224964676292194,864691135752680381,4,7,7677,460.396399
164,176,t,111340327335628821,864691135322954908,4,7,7694,467.373888
165,177,t,110143714885845331,864691135995806410,4,7,7704,455.457412
...,...,...,...,...,...,...,...,...
9483,8521,t,87628053949238190,864691135568173804,7,3,8810,467.945328
9485,7333,t,90791349372223852,864691135118142173,6,6,6657,377.782595
9497,6052,t,91781596964938620,864691135581937773,6,4,8484,530.872297
9501,5907,t,91920685320073392,864691135884234096,4,7,8112,482.932865


In [ ]:
l5t1, l5t2, l5t3, l5t4, l5t5, l5t6, l5t7, l5t8, l5t9, l5t10 = ([] for i in range(10))
for row in range(len(df_layer5)):
    unit_key = {'session': df_layer5.iloc[row]['session'], 'scan_idx': df_layer5.iloc[row]['scan_idx'], 'unit_id': df_layer5.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    l5t1.append(oracle_traces[0].flatten())
    l5t2.append(oracle_traces[1].flatten())
    l5t3.append(oracle_traces[2].flatten())
    l5t4.append(oracle_traces[3].flatten())
    l5t5.append(oracle_traces[4].flatten())
    l5t6.append(oracle_traces[5].flatten())
    l5t7.append(oracle_traces[6].flatten())
    l5t8.append(oracle_traces[7].flatten())
    l5t9.append(oracle_traces[8].flatten())
    l5t10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [ ]:
l5t1= np.array(l5t1)
l5t2= np.array(l5t2)
l5t3= np.array(l5t3)
l5t4= np.array(l5t4)
l5t5= np.array(l5t5)
l5t6= np.array(l5t6)
l5t7= np.array(l5t7)
l5t8= np.array(l5t8)
l5t9= np.array(l5t9)
l5t10= np.array(l5t10)
np.savetxt('layer5_trial1_1_to_2000.csv', l5t1, delimiter= ',')
np.savetxt('layer5_trial2_1_to_2000.csv', l5t2, delimiter= ',')
np.savetxt('layer5_trial3_1_to_2000.csv', l5t3, delimiter= ',')
np.savetxt('layer5_trial4_1_to_2000.csv', l5t4, delimiter= ',')
np.savetxt('layer5_trial5_1_to_2000.csv', l5t5, delimiter= ',')
np.savetxt('layer5_trial6_1_to_2000.csv', l5t6, delimiter= ',')
np.savetxt('layer5_trial7_1_to_2000.csv', l5t7, delimiter= ',')
np.savetxt('layer5_trial8_1_to_2000.csv', l5t8, delimiter= ',')
np.savetxt('layer5_trial9_1_to_2000.csv', l5t9, delimiter= ',')
np.savetxt('layer5_trial10_1_to_2000.csv', l5t10, delimiter= ',')


## Layer 6 Gathering

In [28]:
df_layer6 = df[df['depth'].between(574, 713) & (df['session'] != 9)]
df_layer6

,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,depth
313,332,t,112540169667650845,864691135700243579,5,3,8299,576.947747
314,333,t,109937419486317089,864691136536229026,5,3,8316,596.156823
315,334,t,110289195091748716,864691135385202261,5,3,8321,594.781174
316,335,t,108670576469823157,864691136561660322,5,3,8332,588.433158
317,336,t,105223264254840882,864691136010930339,5,3,8365,602.012822
...,...,...,...,...,...,...,...,...
8668,9037,t,110429657299682398,864691135181774850,5,3,8531,587.096382
9228,2625,t,87702683368468067,864691136309905498,5,3,8600,594.399679
9269,341,t,89602295863921793,864691135799939426,5,3,8486,589.209278
9311,6328,t,110640625690705852,864691135776867424,5,3,8307,583.524655


In [ ]:
l6t1, l6t2, l6t3, l6t4, l6t5, l6t6, l6t7, l6t8, l6t9, l6t10 = ([] for i in range(10))
for row in range(len(df_layer6)):
    unit_key = {'session': df_layer6.iloc[row]['session'], 'scan_idx': df_layer6.iloc[row]['scan_idx'], 'unit_id': df_layer6.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    l6t1.append(oracle_traces[0].flatten())
    l6t2.append(oracle_traces[1].flatten())
    l6t3.append(oracle_traces[2].flatten())
    l6t4.append(oracle_traces[3].flatten())
    l6t5.append(oracle_traces[4].flatten())
    l6t6.append(oracle_traces[5].flatten())
    l6t7.append(oracle_traces[6].flatten())
    l6t8.append(oracle_traces[7].flatten())
    l6t9.append(oracle_traces[8].flatten())
    l6t10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [ ]:
l6t1= np.array(l6t1)
l6t2= np.array(l6t2)
l6t3= np.array(l6t3)
l6t4= np.array(l6t4)
l6t5= np.array(l6t5)
l6t6= np.array(l6t6)
l6t7= np.array(l6t7)
l6t8= np.array(l6t8)
l6t9= np.array(l6t9)
l6t10= np.array(l6t10)
np.savetxt('layer6_trial1_1_to_61.csv', l6t1, delimiter= ',')
np.savetxt('layer6_trial2_1_to_61.csv', l6t2, delimiter= ',')
np.savetxt('layer6_trial3_1_to_61.csv', l6t3, delimiter= ',')
np.savetxt('layer6_trial4_1_to_61.csv', l6t4, delimiter= ',')
np.savetxt('layer6_trial5_1_to_61.csv', l6t5, delimiter= ',')
np.savetxt('layer6_trial6_1_to_61.csv', l6t6, delimiter= ',')
np.savetxt('layer6_trial7_1_to_61.csv', l6t7, delimiter= ',')
np.savetxt('layer6_trial8_1_to_61.csv', l6t8, delimiter= ',')
np.savetxt('layer6_trial9_1_to_61.csv', l6t9, delimiter= ',')
np.savetxt('layer6_trial10_1_to_61.csv', l6t10, delimiter= ',')
